In [1]:
#libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import json

import cwms

### Initializing the database and write access
cwms-python will connect by default to the USACE public database available through https://cwms-data.usace.army.mil/cwms-data/.  This is the database that hosts data for https://water.usace.army.mil/.  The apiRoot can be updated to access data directly from a USACE district database.  This can be done with the cwms.api.init_session.  For users with write capabilitied to the database an apiKey is required and can be also be initialized using the cwms.api.init_sessions.  For users who only want to access data from the public database these steps are not needed and all get functions in cwms-pythong can be used without initializing.

In [2]:
apiRoot = "https://cwms-data-test.cwbi.us/cwms-data/"  

In [3]:
from getpass import getpass
apiKey = "apikey " + getpass()

 ········


In [4]:
api = cwms.api.init_session(api_root=apiRoot,api_key=apiKey)

## Create a Location to Save the Ratings to

In [5]:
location = """
    {
      "name": "TestRating",
      "latitude": 0,
      "longitude": 0,
      "active": true,
      "public-name": "CWMS TESTING",
      "long-name": "CWMS TESTING",
      "description": "CWMS TESTING",
      "timezone-name": "America/Los_Angeles",
      "location-kind": "PROJECT",
      "nation": "US",
      "state-initial": "CA",
      "county-name": "Yolo",
      "nearest-city": "Davis, CA",
      "horizontal-datum": "NAD83",
      "vertical-datum": "NGVD29",
      "elevation": 320.04,
      "bounding-office-id": "MVP",
      "office-id": "MVP"
    }
    """
cwms.api.post(endpoint="locations", data=location, params=None, api_version=2)

## Create A Rating Template and View that template

In [6]:
template_xml = '''<?xml version="1.0" encoding="utf-8"?>
<rating-template office-id="MVP">
<parameters-id>Stage;Flow</parameters-id>
<version>USGS-EXSA-TEST</version>\n  <ind-parameter-specs>
   <ind-parameter-spec position="1">
    <parameter>Stage</parameter>
    <in-range-method>LINEAR</in-range-method>
    <out-range-low-method>LINEAR</out-range-low-method>
    <out-range-high-method>LINEAR</out-range-high-method>
   </ind-parameter-spec>
  </ind-parameter-specs>
  <dep-parameter>Flow</dep-parameter>
  <description>Expanded, Shift-Adjusted Stream Rating</description>
 </rating-template>'''

In [7]:
#store a rating template
cwms.store_rating_template(data = template_xml)

In [8]:
# get a individual rating template
template = cwms.get_rating_template(template_id='Stage;Flow.USGS-EXSA-TEST',office_id='MVP')

In [9]:
template.json

{'office-id': 'MVP',
 'id': 'Stage;Flow.USGS-EXSA-TEST',
 'version': 'USGS-EXSA-TEST',
 'description': 'Expanded, Shift-Adjusted Stream Rating',
 'dependent-parameter': 'Flow',
 'independent-parameter-specs': [{'parameter': 'Stage',
   'in-range-method': 'LINEAR',
   'out-range-low-method': 'LINEAR',
   'out-range-high-method': 'LINEAR'}],
 'rating-ids': []}

In [10]:
template.df

,office-id,id,version,description,dependent-parameter,independent-parameter-specs,rating-ids
0,MVP,Stage;Flow.USGS-EXSA-TEST,USGS-EXSA-TEST,"Expanded, Shift-Adjusted Stream Rating",Flow,"[{'parameter': 'Stage', 'in-range-method': 'LI...",[]


In [11]:
# get all rating templates in the database for an office
templates = cwms.get_rating_templates(office_id = "MVP")

In [12]:
# get all rating templates that have Stage;Flow
templates_mask = cwms.get_rating_templates(office_id = "MVP", template_id_mask = "Stage;Flow*")

In [13]:
templates_mask.df

,office-id,id,version,dependent-parameter,independent-parameter-specs,rating-ids,description
0,MVP,Stage;Flow.Linear,Linear,Flow,"[{'parameter': 'Stage', 'in-range-method': 'LI...",[JAHM5.Stage;Flow.Linear.Step],NaN
1,MVP,Stage;Flow.Log,Log,Flow,"[{'parameter': 'Stage', 'in-range-method': 'LO...","[BROM5.Stage;Flow.Log.Step, BeaverFalls.Stage;...",NaN
2,MVP,Stage;Flow.Standard,Standard,Flow,"[{'parameter': 'Stage', 'in-range-method': 'LI...",[EauGalle_Dam-MorningGlory.Stage;Flow.Standard...,NaN
3,MVP,Stage;Flow.USGS-BASE,USGS-BASE,Flow,"[{'parameter': 'Stage', 'in-range-method': 'LO...","[ABRN8.Stage;Flow.USGS-BASE.USGS-NWIS, AGYM5.S...",Stream Rating (Base + Shifts and Offsets)
4,MVP,Stage;Flow.USGS-EXSA,USGS-EXSA,Flow,"[{'parameter': 'Stage', 'in-range-method': 'LI...","[ABRN8.Stage;Flow.USGS-EXSA.USGS-NWIS, AGYM5.S...","Expanded, Shift-Adjusted Stream Rating"
5,MVP,Stage;Flow.USGS-EXSA-TEST,USGS-EXSA-TEST,Flow,"[{'parameter': 'Stage', 'in-range-method': 'LI...",[],"Expanded, Shift-Adjusted Stream Rating"


## Create a Rating Specification and View that Specification

In [14]:
spec_xml = '''<?xml version="1.0" encoding="utf-8"?>
<rating-spec office-id="MVP">
  <rating-spec-id>TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST</rating-spec-id>
  <template-id>Stage;Flow.USGS-EXSA-TEST</template-id>
  <location-id>TestRating</location-id>
  <version>USGS-NWIS-TEST</version>
  <source-agency>USGS</source-agency>
  <in-range-method>LINEAR</in-range-method>
  <out-range-low-method>NEAREST</out-range-low-method>
  <out-range-high-method>NEAREST</out-range-high-method>
  <active>true</active>
  <auto-update>true</auto-update>
  <auto-activate>true</auto-activate>
  <auto-migrate-extension>true</auto-migrate-extension>
  <ind-rounding-specs>
   <ind-rounding-spec position="1">2223456782</ind-rounding-spec>
  </ind-rounding-specs>
  <dep-rounding-spec>2222233332</dep-rounding-spec>
  <description>TESTing rating spec</description>
 </rating-spec>'''

In [16]:
cwms.store_rating_spec(data=spec_xml)

In [17]:
#get a single rating specification
rating_spec = cwms.get_rating_spec(rating_id='TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST',office_id='MVP')

In [18]:
rating_spec.json

{'office-id': 'MVP',
 'rating-id': 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST',
 'template-id': 'Stage;Flow.USGS-EXSA-TEST',
 'location-id': 'TestRating',
 'version': 'USGS-NWIS-TEST',
 'source-agency': 'USGS',
 'in-range-method': 'LINEAR',
 'out-range-low-method': 'NEAREST',
 'out-range-high-method': 'NEAREST',
 'active': True,
 'auto-update': True,
 'auto-activate': True,
 'auto-migrate-extension': True,
 'independent-rounding-specs': [{'value': '2223456782'}],
 'dependent-rounding-spec': '2222233332',
 'description': 'TESTing rating spec'}

In [19]:
rating_spec.df

,office-id,rating-id,template-id,location-id,version,source-agency,in-range-method,out-range-low-method,out-range-high-method,active,auto-update,auto-activate,auto-migrate-extension,independent-rounding-specs,dependent-rounding-spec,description
0,MVP,TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS...,Stage;Flow.USGS-EXSA-TEST,TestRating,USGS-NWIS-TEST,USGS,LINEAR,NEAREST,NEAREST,True,True,True,True,[{'value': '2223456782'}],2222233332,TESTing rating spec


In [20]:
#get multiple rating specs
rating_specs = cwms.get_rating_specs(office_id="MVP", rating_id_mask="*USGS-EXSA.USGS-NWIS")

In [21]:
rating_specs.df

,office-id,rating-id,template-id,location-id,version,in-range-method,out-range-low-method,out-range-high-method,active,auto-update,auto-activate,auto-migrate-extension,independent-rounding-specs,dependent-rounding-spec,description,effective-dates,source-agency
0,MVP,ABRN8.Stage;Flow.USGS-EXSA.USGS-NWIS,Stage;Flow.USGS-EXSA,ABRN8,USGS-NWIS,LINEAR,NEAREST,NEAREST,True,True,True,True,[{'value': '2223456782'}],2222233332,"WILD RICE RIVER NR ABERCROMBIE, ND Expanded, S...","[2014-11-04T21:15:00Z, 2014-11-04T21:15:00Z, 2...",NaN
1,MVP,05053000.Stage;Flow.USGS-EXSA.USGS-NWIS,Stage;Flow.USGS-EXSA,05053000,USGS-NWIS,LINEAR,NEAREST,NEAREST,True,True,True,True,[{'value': '2223456782'}],2222233332,"WILD RICE RIVER NR ABERCROMBIE, ND Expanded, S...","[2014-11-04T21:15:00Z, 2015-09-03T18:20:00Z, 2...",NaN
2,MVP,05087500.Stage;Flow.USGS-EXSA.USGS-NWIS,Stage;Flow.USGS-EXSA,05087500,USGS-NWIS,LINEAR,NEAREST,NEAREST,True,True,True,True,[{'value': '2223456782'}],2222233332,"MIDDLE RIVER AT ARGYLE, MN Expanded, Shift-Adj...","[2014-10-17T15:48:00Z, 2015-04-29T17:00:00Z, 2...",NaN
3,MVP,AGYM5.Stage;Flow.USGS-EXSA.USGS-NWIS,Stage;Flow.USGS-EXSA,AGYM5,USGS-NWIS,LINEAR,NEAREST,NEAREST,True,True,True,True,[{'value': '2223456782'}],2222233332,"MIDDLE RIVER AT ARGYLE, MN Expanded, Shift-Adj...","[2014-10-17T15:48:00Z, 2014-10-17T15:48:00Z, 2...",NaN
4,MVP,AKRN8.Stage;Flow.USGS-EXSA.USGS-NWIS,Stage;Flow.USGS-EXSA,AKRN8,USGS-NWIS,LINEAR,NEAREST,NEAREST,True,True,True,True,[{'value': '2223456782'}],2222233332,"TONGUE RIVER AT AKRA, ND Expanded, Shift-Adjus...","[2014-10-21T16:45:00Z, 2014-10-21T16:45:00Z, 2...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,MVP,MN00577-Tailwater.Stage;Stage-Shift.USGS-EXSA....,Stage;Stage-Shift.USGS-EXSA,MN00577-Tailwater,USGS-NWIS,LINEAR,LINEAR,NEAREST,True,True,True,False,[{'value': '2223456782'}],2223456782,USGS-style rating shifts,[2023-02-17T21:41:00Z],NaN
286,MVP,WHRM5-Tailwater.Stage;Stage-Shift.USGS-EXSA.US...,Stage;Stage-Shift.USGS-EXSA,WHRM5-Tailwater,USGS-NWIS,LINEAR,LINEAR,NEAREST,True,True,True,False,[{'value': '2223456782'}],2223456782,USGS-style rating shifts,[2023-02-17T21:41:00Z],NaN
287,MVP,TraverseWR_Dam-Tailwater.Stage;Stage-Shift.USG...,Stage;Stage-Shift.USGS-EXSA,TraverseWR_Dam-Tailwater,USGS-NWIS,LINEAR,LINEAR,NEAREST,True,True,True,False,[{'value': '2223456782'}],2223456782,USGS-style rating shifts,[2023-02-17T21:41:00Z],NaN
288,MVP,05050000.Stage;Stage-Shift.USGS-EXSA.USGS-NWIS,Stage;Stage-Shift.USGS-EXSA,05050000,USGS-NWIS,LINEAR,LINEAR,NEAREST,True,True,True,False,[{'value': '2223456782'}],2223456782,USGS-style rating shifts,[2023-02-17T21:41:00Z],NaN


## Adding a Rating to the Rating Spec.  
#### Rating table is first created as a dataframe with ind and dep values.
#### Rating Table df is then converted to a json and saved in the database. 
#### Two rating curves are saved with different effective dates


In [22]:
rating_table = pd.DataFrame({'ind' : [1,2,3,4,5,6,7,8,9,10], 'dep': [100,200,300,400,500,600,700,800,900,1000]})

In [23]:
rating_table

,ind,dep
0,1,100
1,2,200
2,3,300
3,4,400
4,5,500
5,6,600
6,7,700
7,8,800
8,9,900
9,10,1000


In [24]:
rating_table_json = cwms.rating_simple_df_to_json(
    data = rating_table,
    rating_id = 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST',
    office_id = 'MVP',
    units = 'ft;cfs',
    effective_date = datetime.now(),
    transition_start_date = None,
    description = 'Testing Rating Curves.  Test Rating'
)

In [25]:
rating_table_json

{'noNamespaceSchemaLocation': 'http://www.hec.usace.army.mil/xmlSchema/cwms/Ratings.xsd',
 'rating-template': {'office-id': 'MVP',
  'parameters-id': 'Stage;Flow',
  'version': 'USGS-EXSA-TEST',
  'ind-parameter-specs': {'ind-parameter-spec': {'position': '1',
    'parameter': 'Stage',
    'in-range-method': 'LINEAR',
    'out-range-low-method': 'LINEAR',
    'out-range-high-method': 'LINEAR'}},
  'dep-parameter': 'Flow',
  'description': 'Expanded, Shift-Adjusted Stream Rating'},
 'rating-spec': {'office-id': 'MVP',
  'rating-spec-id': 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST',
  'template-id': 'Stage;Flow.USGS-EXSA-TEST',
  'location-id': 'TestRating',
  'version': 'USGS-NWIS-TEST',
  'source-agency': 'USGS',
  'in-range-method': 'LINEAR',
  'out-range-low-method': 'NEAREST',
  'out-range-high-method': 'NEAREST',
  'active': 'true',
  'auto-update': 'true',
  'auto-activate': 'true',
  'auto-migrate-extension': 'true',
  'ind-rounding-specs': {'ind-rounding-spec': {'posit



#### Once the rating table is converted to JSON it can be stored into the database

In [26]:
cwms.update_ratings(data = rating_table_json, rating_id = 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST')

#### Create a second rating table with a different effective date

In [27]:
rating_table.dep = rating_table.dep/2

In [28]:
rating_table

,ind,dep
0,1,50.0
1,2,100.0
2,3,150.0
3,4,200.0
4,5,250.0
5,6,300.0
6,7,350.0
7,8,400.0
8,9,450.0
9,10,500.0


In [29]:
rating_table_json = cwms.rating_simple_df_to_json(
    data = rating_table,
    rating_id = 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST',
    office_id = 'MVP',
    units = 'ft;cfs',
    effective_date = datetime.now(timezone.utc) - timedelta(days = 50),
    transition_start_date = None,
    description = 'Testing Rating Curves.  Test Rating'
)

In [30]:
cwms.update_ratings(data = rating_table_json, rating_id = 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST')

#### Now we can see both rating tables attached to the rating spec

In [31]:
ratings = cwms.get_ratings(rating_id = 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST', office_id = 'MVP', method = 'EAGER')

In [32]:
ratings.df

,office-id,rating-spec-id,units-id,effective-date,transition-start-date,create-date,active,description,rating-points.point
0,MVP,TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS...,ft;cfs,2024-05-22T19:29:00Z,,2024-07-11T19:28:00Z,true,Testing Rating Curves. Test Rating,"[{'ind': '1.0', 'dep': '50.0'}, {'ind': '2.0',..."
1,MVP,TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS...,ft;cfs,2024-07-11T12:29:00Z,,2024-07-11T19:28:00Z,true,Testing Rating Curves. Test Rating,"[{'ind': '1.0', 'dep': '100.0'}, {'ind': '2.0'..."


#### We Can use the function get_current_rating to grab only the rating curve with the latest effective date that is currently active.

In [33]:
current_rating = cwms.get_current_rating(rating_id = 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST', office_id = 'MVP')

In [34]:
current_rating.df

,ind,dep
0,1.0,100.0
1,2.0,200.0
2,3.0,300.0
3,4.0,400.0
4,5.0,500.0
5,6.0,600.0
6,7.0,700.0
7,8.0,800.0
8,9.0,900.0
9,10.0,1000.0


In [35]:
current_rating.json

{'noNamespaceSchemaLocation': 'http://www.hec.usace.army.mil/xmlSchema/cwms/Ratings.xsd',
 'rating-template': {'office-id': 'MVP',
  'parameters-id': 'Stage;Flow',
  'version': 'USGS-EXSA-TEST',
  'ind-parameter-specs': {'ind-parameter-spec': {'position': '1',
    'parameter': 'Stage',
    'in-range-method': 'LINEAR',
    'out-range-low-method': 'LINEAR',
    'out-range-high-method': 'LINEAR'}},
  'dep-parameter': 'Flow',
  'description': 'Expanded, Shift-Adjusted Stream Rating'},
 'rating-spec': {'office-id': 'MVP',
  'rating-spec-id': 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST',
  'template-id': 'Stage;Flow.USGS-EXSA-TEST',
  'location-id': 'TestRating',
  'version': 'USGS-NWIS-TEST',
  'source-agency': 'USGS',
  'in-range-method': 'LINEAR',
  'out-range-low-method': 'NEAREST',
  'out-range-high-method': 'NEAREST',
  'active': 'true',
  'auto-update': 'true',
  'auto-activate': 'true',
  'auto-migrate-extension': 'true',
  'ind-rounding-specs': {'ind-rounding-spec': {'posit

### Delete Ratings
#### Delete Rating table from Rating Spec based on data range

In [36]:
cwms.delete_ratings(
    rating_id='TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST', 
    office_id='MVP', 
    begin = datetime.now(timezone.utc) - timedelta(days = 52),
    end = datetime.now(timezone.utc) - timedelta(days = 48))

### Now only one rating is present in the rating spec

In [37]:
ratings = cwms.get_ratings(rating_id = 'TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST', office_id = 'MVP', method = 'LAZY')

In [38]:
ratings.df

,office-id,rating-spec-id,units-id,effective-date,transition-start-date,create-date,active,description
0,MVP,TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS...,ft;cfs,2024-07-11T12:29:00Z,,2024-07-11T19:28:00Z,true,Testing Rating Curves. Test Rating


#### Delete Rating Spec, Rating Template, and Location

In [39]:
cwms.delete_rating_spec(rating_id='TestRating.Stage;Flow.USGS-EXSA-TEST.USGS-NWIS-TEST', office_id='MVP', delete_method='DELETE_ALL')

In [40]:
cwms.delete_rating_template(template_id='Stage;Flow.USGS-EXSA-TEST',office_id='MVP',delete_method='DELETE_ALL')

In [41]:
cwms.api.delete(endpoint=f"locations/TestRating", params = {'office':'MVP'})